In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import math
import os

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

In [3]:
files_path = os.path.join("..", "data", "maccs")

dataset_files = [os.path.join(files_path, file) for file in os.listdir(files_path)]

data_path = dataset_files[0]
print(f"Read from: {data_path}")

df = pd.read_csv(data_path)
df.head()

Read from: ../data/maccs/Ames_MACCS.csv


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x158,x159,x160,x161,x162,x163,x164,x165,x166,y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,2.0,1.0,42.0,6.0,44.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,0.0,4.0,0.0,6.0,2.0,6.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,1.0,4.0,0.0,6.0,3.0,6.0,0.0,1.0


In [4]:
# Find unique values for each feature
dfX = df.drop(["y"], axis=1)
col_redundant = np.where(dfX.nunique() == 1)[0]
print(col_redundant)

# Remove columns that do not have useful information
dfX = dfX.iloc[:, np.where(dfX.nunique() > 1)[0]]
dfX.head()

[  0   1   3   4   5   6   8   9  11  34  43 165]


,x3,x8,x11,x13,x14,x15,x16,x17,x18,x19,...,x156,x157,x158,x159,x160,x161,x162,x163,x164,x165
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,5.0,0.0,2.0,1.0,42.0,6.0,44.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,2.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,2.0,1.0,0.0,4.0,0.0,6.0,2.0,6.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,2.0,1.0,0.0,4.0,0.0,0.0,2.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,2.0,1.0,4.0,0.0,6.0,3.0,6.0


In [5]:
def nCr(n, r):
    f = math.factorial
    return f(n) / f(r) / f(n - r)


nCr(153, math.floor(0.05 * 153))

338795791740.0

In [6]:
# Find valued in the column which has most distinct values
unique_values = dfX.nunique()
# The column that holds the most distinc values
idx_max = unique_values.argmax()

columns = dfX.columns
print(columns[idx_max])
print(idx_max)
# The most distinc values
print(unique_values.iloc[idx_max])

x165
153
47


In [7]:
# Show all distinct values on column 153
np.unique(dfX.iloc[:, idx_max].to_numpy())

array([ 0.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
       15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
       28., 29., 30., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
       42., 43., 44., 45., 48., 52., 69., 71.])

In [8]:
X = dfX.to_numpy()
X.shape

(6512, 154)

In [9]:
X_test = X[:10]
X_test.shape

(10, 154)

In [10]:
# Molecular Similarity Measure
X_dot = np.dot(X_test, X_test.T)
denominator = np.sum(X_test + X_test, axis=1) - X_dot
sim = X_dot / (denominator + 1e-9)
print(np.round(sim, 2))

[[-1.080e+00  8.400e-01 -1.320e+00  1.260e+00 -1.490e+00  3.310e+00
  -1.420e+00 -1.160e+00 -1.370e+00 -1.140e+00]
 [ 1.800e-01 -5.120e+00  3.290e+00 -5.950e+00  1.480e+00  4.330e+00
   1.780e+00  9.000e-02  1.890e+00  7.900e-01]
 [-3.250e+00  4.000e+00 -3.470e+00  7.670e+00  4.300e+00  1.600e+00
   8.370e+00  3.830e+00 -6.570e+00 -4.130e+00]
 [ 1.600e-01  8.330e+00  1.700e+00  1.040e+11  8.300e-01  1.950e+00
   9.300e-01  9.000e-02  9.000e-01  5.100e-01]
 [-3.260e+00  5.090e+00 -1.223e+01  5.930e+00 -3.580e+00  5.590e+00
  -3.440e+00 -9.433e+01  3.730e+00 -3.350e+00]
 [ 2.600e-01  1.860e+00  9.000e-01  2.470e+00  9.400e-01 -8.000e+00
   1.070e+00  3.300e-01  3.700e-01  5.000e-01]
 [-3.260e+00  4.380e+00 -1.223e+01  4.780e+00 -4.560e+00  4.600e+00
  -3.660e+00  1.233e+01  3.730e+00 -3.350e+00]
 [-1.260e+00  2.000e-01 -2.920e+00  2.800e-01 -3.250e+00  1.670e+00
  -3.200e+00 -1.410e+00 -5.330e+00 -1.520e+00]
 [-3.650e+00  2.680e+00 -4.840e+00  2.470e+00  1.680e+00  6.000e-01
   2.240e+00

In [11]:
# Sim for 1st sample
np.sum(X_test[0] * X_test[0]) / (
    np.sum(X_test[0]) + np.sum(X_test[0]) - np.sum(X_test[0] * X_test[0])
)

-1.0808394869801787

In [12]:
np.random.randint(0, 2, size=(10, 3))

array([[0, 1, 1],
       [0, 0, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 0],
       [1, 0, 0],
       [0, 0, 0],
       [1, 0, 1],
       [0, 1, 1],
       [0, 0, 1]])

In [13]:
def category2code(df):
    """Change all columns to ordinal data, represent in codes.
    (e.g.: from {2, 5, 8} to {0, 1, 2})
    """
    result = df.copy()
    for c in df.columns:
        categories = np.sort(np.unique(df[c].to_numpy()))
        cattype = CategoricalDtype(categories=categories, ordered=True)
        result[c] = result[c].astype(cattype).cat.codes
    return result

In [14]:
my_dfX = category2code(dfX)
my_dfX.head()

,x3,x8,x11,x13,x14,x15,x16,x17,x18,x19,...,x156,x157,x158,x159,x160,x161,x162,x163,x164,x165
0,0,0,0,0,0,0,0,0,0,0,...,4,0,0,5,0,2,1,29,6,41
1,0,0,0,0,0,0,0,0,0,0,...,2,0,2,1,0,5,0,0,2,0
2,0,0,0,0,0,0,0,0,0,0,...,3,0,2,1,0,4,0,1,2,4
3,0,0,0,0,0,0,0,0,0,0,...,2,0,2,1,0,4,0,0,2,0
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,2,1,4,0,1,3,4


## Generate Random Samples on Categorical Data
1. Change all columns to ordinal data, represent in codes (e.g.: from {2, 5, 8} to {0, 1, 2})
1. Find min and max for each column
1. Find the indices want to add noise (5% of 100 attributes mean we choose 5 indices)
1. Generate a list of {-1, 1} with same length of the indices to represent noise 
1. Added noise to the sample
1. Clip the sample

In [68]:
def get_range(X):
    if isinstance(X, pd.DataFrame):
        return (X.min(axis=0).to_numpy(), X.max(axis=0).to_numpy())
    return (X.min(axis=0), X.max(axis=0))

In [100]:
def generate_random_int_samples(x, x_min, x_max, r, size):
    """Generate random samples around integer value x, bounded by Manhattan
    distance [-1, 1].
    """
    dtype = x.dtype
    n_features = len(x)
    shape = (size, int(np.floor(r * n_features)))
    # Find the indices want to add noise (5% of 100 attributes mean we choose 5 indices)
    indices = np.random.choice(np.arange(n_features), size=shape, replace=False)
    # Generate a list of {-1, 1} with same length of the indices to represent noise 
    noise = np.random.randint(0, 2, size=shape).astype(dtype)
    noise[noise == 0] = -1
    # noise = np.ones(shape=shape).astype(dtype)
    rng_samples = np.repeat([x], repeats=size, axis=0)
    # Added noise to the sample
    for i_sample, i_feature in enumerate(indices):
        rng_samples[i_sample, i_feature] += noise[i_sample]
    # To handle the situation where the majority of a sparse matrix only contain 0.
    rng_samples[rng_samples < 0] = 1
    # Ensure the random samples are valid.
    rng_samples = np.clip(rng_samples, a_min=x_min, a_max=x_max)
    return rng_samples


X = my_dfX.to_numpy()
x_min, x_max = get_range(X)
rng_samples = generate_random_int_samples(X[0], x_min, x_max, r=0.1, size=5)
rng_samples.shape

(5, 154)

In [101]:
X[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,
        4,  0,  0,  0,  0,  0,  0,  0, 28,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  2,  0,  0,  0,  1,  0,  5,  0,  0,  0,
        1,  0,  0,  0,  0,  1,  2,  0,  0,  3,  0,  1,  6,  5,  1,  4,  0,
        0,  0,  0,  2,  6,  0,  6,  0,  4,  0,  0,  5,  0,  2,  1, 29,  6,
       41], dtype=int8)

In [102]:
rng_samples

array([[ 0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
         1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         1,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  0,  0,  0,  2,  1,  0,  0,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  5,  3,  0,  0,  0,  0,  1,  0,  1, 28,  1,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1,  2,  0,  0,
         0,  1,  0,  5,  0,  0,  0,  1,  0,  0,  0,  0,  1,  2,  0,  0,
         3,  0,  1,  6,  5,  1,  3,  0,  0,  0,  0,  2,  6,  0,  6,  0,
         4,  0,  0,  5,  0,  2,  1, 29,  6, 41],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  1,  1,  0,  0,  2,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         1,  0,

In [103]:
print(0.1 * len(X[0]))
print(np.sum(rng_samples != X[0], axis=1))

15.4
[14 15 15 15 14]
